# LCR 111. 除法求值 (medium)

给定一个变量对数组 equations 和一个实数值数组 values 作为已知条件，其中 equations[i] = [Ai, Bi] 和 values[i] 共同表示等式 Ai / Bi = values[i] 。每个 Ai 或 Bi 是一个表示单个变量的字符串。

另有一些以数组 queries 表示的问题，其中 queries[j] = [Cj, Dj] 表示第 j 个问题，请你根据已知条件找出 Cj / Dj = ? 的结果作为答案。

返回 所有问题的答案 。如果存在某个无法确定的答案，则用 -1.0 替代这个答案。如果问题中出现了给定的已知条件中没有出现的字符串，也需要用 -1.0 替代这个答案。

注意：输入总是有效的。可以假设除法运算中不会出现除数为 0 的情况，且不存在任何矛盾的结果。

示例 1：
```
输入：equations = [["a","b"],["b","c"]], values = [2.0,3.0], queries = [["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]]
输出：[6.00000,0.50000,-1.00000,1.00000,-1.00000]
解释：
条件：a / b = 2.0, b / c = 3.0
问题：a / c = ?, b / a = ?, a / e = ?, a / a = ?, x / x = ?
结果：[6.0, 0.5, -1.0, 1.0, -1.0 ]
```
示例 2：
```
输入：equations = [["a","b"],["b","c"],["bc","cd"]], values = [1.5,2.5,5.0], queries = [["a","c"],["c","b"],["bc","cd"],["cd","bc"]]
输出：[3.75000,0.40000,5.00000,0.20000]
```
示例 3：
```
输入：equations = [["a","b"]], values = [0.5], queries = [["a","b"],["b","a"],["a","c"],["x","y"]]
输出：[0.50000,2.00000,-1.00000,-1.00000]
``` 

提示：
```
1 <= equations.length <= 20
equations[i].length == 2
1 <= Ai.length, Bi.length <= 5
values.length == equations.length
0.0 < values[i] <= 20.0
1 <= queries.length <= 20
queries[i].length == 2
1 <= Cj.length, Dj.length <= 5
Ai, Bi, Cj, Dj 由小写英文字母与数字组成
``` 

注意：本题与主站 399 题相同： https://leetcode-cn.com/problems/evaluate-division/

In [1]:
# 1. UF 并查集
from typing import List
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        ID, word_ID = 0, {}
        for w1, w2 in equations:
            if w1 not in word_ID:
                word_ID[w1] = ID
                ID += 1
            if w2 not in word_ID:
                word_ID[w2] = ID
                ID += 1

        UF = UnionFind(ID)
        for [w1, w2], val in zip(equations, values):
            x_ID, y_ID = word_ID[w1], word_ID[w2]
            UF.union(x_ID, y_ID, val)

        ans = []
        for w1, w2 in queries:
            # 默认不在字典中，无公共父节点，tmp=-1
            tmp = -1.0
            if w1 in word_ID  and w2 in word_ID:
                x_ID, y_ID = word_ID[w1], word_ID[w2]
                fx, fy = UF.find(x_ID), UF.find(y_ID)
                # 有公共父节点，权重相除
                if fx == fy:
                    tmp = UF.w[x_ID] / UF.w[y_ID]
            ans.append(tmp)
        return ans
        

class UnionFind:
    def __init__(self, n: int):
        self.f = list(range(n))
        self.w = [1.0] * n

    def find(self, x: int) -> int:
        if self.f[x] != x:
            father = self.find(self.f[x])
            # w[x] = v[x] / v[father] = v[x] / v[f[x]] * v[f[x]] / v[father] = w[x] * w[f[x]]
            self.w[x] *= self.w[self.f[x]]
            self.f[x] = father
        return self.f[x]

    def union(self, x: int, y: int, v: float):
        fx = self.find(x)
        fy = self.find(y)
        self.f[fx] = fy
        # w[fx] = v[fx] / v[fy] = (v[x] / w[x]) / (v[y] / w[y]) = v[x] / v[y] * w[y] / w[x]
        self.w[fx] = v * self.w[y] / self.w[x]


print(Solution().calcEquation([["a","b"],["b","c"]], [2.0,3.0], [["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]]))
print(Solution().calcEquation([["a","b"],["b","c"],["bc","cd"]], [1.5,2.5,5.0], [["a","c"],["c","b"],["bc","cd"],["cd","bc"]]))
print(Solution().calcEquation([["a","b"]], [0.5], [["a","b"],["b","a"],["a","c"],["x","y"]]))

[6.0, 0.5, -1.0, 1.0, -1.0]
[3.75, 0.4, 5.0, 0.2]
[0.5, 2.0, -1.0, -1.0]


In [2]:
# BFS
from typing import List
from collections import defaultdict
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        graph = defaultdict(list)
        allnum = set()
        for (u, v), w in zip(equations, values):
            graph[u].append((v, w))
            graph[v].append((u, 1/w))
            allnum.update((u, v))
        def bfs(st, ed):
            if st not in allnum or ed not in allnum:
                return -1.0
            q = [(st, 1.0)]
            see = set([st])
            while q:
                u, val = q.pop(0)
                if u == ed:
                    return val # 一定要在这里返回，不然会超时
                for v, w in graph[u]:
                    if v not in see:
                        q.append((v, val * w))
                        see.add(v)
            return -1.0
        return [bfs(u, v) for u, v in queries]

print(Solution().calcEquation([["a","b"],["b","c"]], [2.0,3.0], [["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]]))
print(Solution().calcEquation([["a","b"],["b","c"],["bc","cd"]], [1.5,2.5,5.0], [["a","c"],["c","b"],["bc","cd"],["cd","bc"]]))
print(Solution().calcEquation([["a","b"]], [0.5], [["a","b"],["b","a"],["a","c"],["x","y"]]))

[6.0, 0.5, -1.0, 1.0, -1.0]
[3.75, 0.4, 5.0, 0.2]
[0.5, 2.0, -1.0, -1.0]
